# City Test appointment ETL

Download data from Form.io and Acuity, combine and send to Color

1. Get some data from form.io
2. Get some data from Acuity
3. Combine it to the right format

## Data that's needed
 - DSW# - acuity form thing - form ID = 1368026 , field id = 7514073
 - Last - acuity top level
 - First - acuity top level
 - Email - acuity top level
 - Phone - acuity top level
 - Test Date - acuity top level
 - Test Time - acuity top level
 - insurance/pcp - need to understand how it's formatted - from form.io. 
     - insurance (including other) - top level
     - primary care first name - if PCP key present
     - primary care last name - if PCP key present
     - practice or hospital - ditto
     - city - ditto 
     - state - ditto

In [38]:
pip install paramiko

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [39]:
import requests
from requests.auth import HTTPBasicAuth
from urllib.parse import urljoin
import datetime
from datetime import date
import csv
import pysftp
import paramiko

# Acuity

In [6]:
def get_acuity_appointments(start_date=date.today(), end_date=date.today()):
    url = urljoin(ACUITY_BASE_URL, '/api/v1/appointments')
    params={
        'appointmentTypeID': ACUITY_APPT_TYPE,
        'minDate': start_date.strftime('%Y-%m-%d'),
        'maxDate': end_date.strftime('%Y-%m-%d'),
        'canceled': False
    }
    
    
    response = requests.get(
        url,
        auth=HTTPBasicAuth(ACUITY_USER, ACUITY_PWD),
        params=params
    )
    
    response.raise_for_status()
    
    return response.json()

In [7]:
appointments = get_acuity_appointments((date.today() + datetime.timedelta(days=2)), (date.today() + datetime.timedelta(days=2)))

# Form.io

In [83]:
def get_formio_responses(dsw_ids, select_fields = 'data.dsw,data.insuranceCarrier,data.hasPCP,data.pcp,created,data.lastReportedWorkDate'):
    headers = {
        'x-token': '{}'.format(FORMIO_API_KEY),
        'Content-Type': 'application/json'
    }
    formio_url = '{base_url}/form/{form_id}/{submission_endpoint}'.format(
        base_url=FORMIO_BASE_URL, 
        form_id=FORM_ID, 
        submission_endpoint='submission'
    )
    
    # FIXME - set limit based on length of dsws and limit length of dsws to max 2048 query string
    params = {'limit': 500}
    if select_fields: params['select'] = select_fields
    if dsw_ids: params['data.dsw__in'] = ','.join(dsw_ids)

    response = requests.get(
        formio_url,
        headers=headers,
        params=params
    )
    response.raise_for_status()
    return response.json()

In [84]:
resps = get_formio_responses(dsws, select_fields=None)

In [85]:
len(resps)

43

# Combine the two and format it nicely
1. ~~Get appointments from Acuity~~
1. ~~Get the fields we want and the format that we want~~
1. ~~Get DSWs from appointments~~
1. ~~Query for Insurance/PCP data from Form.io based on DSWs~~
1. ~~Combine the data on DSW~~
1. Format it nicely

In [14]:
appointments = get_acuity_appointments((date.today() + datetime.timedelta(days=-1)), (date.today() + datetime.timedelta(days=-1)))

In [15]:
def parse_appointment(appointment):
    dsw_field_id = 7514073
    dsw_form_id = 1368026
    top_level_fields = [
        'firstName',
        'lastName',
        'phone',
        'email',
        'datetime',
        'id'
    ]
    # Filter to only keys that we want
    parsed = dict((k, appointment[k]) for k in top_level_fields if k in appointment)

    
    # Looks like next is the most performant way to do this search https://stackoverflow.com/questions/8653516/python-list-of-dictionaries-search
    dsw_form = next((item.get('values') for item in appointment['forms'] if item['id'] == dsw_form_id), [])
    dsw = next((item.get('value') for item in dsw_form if item['fieldID'] == dsw_field_id), None)
    
    
    parsed['dsw'] = dsw if dsw else None
    
    # Rename id and datetime
    parsed['acuityId'] = parsed.pop('id')
    parsed['appointmentDatetime'] = parsed.pop('datetime')

    return parsed
    
    

In [16]:
parse_appointment(appointments[0])

{'firstName': 'pavina',
 'lastName': 'marin',
 'phone': '5555555555',
 'email': 'bobo@dogo.com',
 'dsw': '1111111',
 'acuityId': 370386671,
 'appointmentDatetime': '2020-04-07T16:00:00-0700'}

In [17]:
parsed_appointments = [parse_appointment(appt) for appt in appointments]

In [126]:
def parse_formio_response(response):
    dsw = response['data']['dsw']
    parsed = {
        'hasNoPCP': response['data'].get('hasPCP', None),
        'responseSubmittedDate': response['created'],
        'lastReportedWorkDate': response['data']['lastReportedWorkDate'],
        'insuranceCarrier': response['data']['insuranceCarrier'],
        'formioId': response['_id']
    }    
    if ('pcp' in response['data']): parsed.update(response['data']['pcp']) 
    
    return (dsw, parsed)

def merge_acuity_formio(parsed_appointments, parsed_responses):
    merged = []
    for appointment in parsed_appointments:
        dsw = appointment['dsw']
        formio_merge = parsed_responses.get(dsw, {}) if dsw else {}
        if !formio_merge: print('Missing formio data for DSW:', dsw)
        merged.append({**appointment, **formio_merge})
    return merged
    

SyntaxError: invalid syntax (<ipython-input-126-0ec961f203f2>, line 19)

In [128]:
# Convert to csv
def write_csv(toCSV, filename='./testOutput.csv'):
    all_keys = [
        'firstName',
        'lastName',
        'phone',
        'email',
        'dsw',
        'appointmentDatetime',
        'lastReportedWorkDate',
        'insuranceCarrier',
        'hasNoPCP',
        'pcpFirstName',
        'pcpLastName',
        'pcpPractice',
        'pcpCity',
        'pcpState',
        'iAuthorizeColorToShareMyInformationAndTestResultsWithThePrimaryCarePhysicianIHaveIdentifiedOnThisFormForTreatmentAndCarePurposes',
        'formioId',
        'responseSubmittedDate',
        'acuityId'
    ]
    with open(filename, 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=all_keys, restval='')
        dict_writer.writeheader()
        dict_writer.writerows(toCSV)
    return filename

# Upload to FTP server

In [35]:
# How to upload to FTP

files = {'file': (filename, open(filename, 'rb'), 'text/plain', {'Expires': '0'})}

headers = {
  'ACCESS_KEY': SFDS_SFTP_ACCESS_KEY,
  'X-SFTP-HOST': SFTP_HOSTNAME,
  'X-SFTP-HOST-KEY': SFTP_HOST_KEY,
  'X-SFTP-USER': SFTP_USERNAME,
  'X-SFTP-PASSWORD': SFTP_PASSWORD,
  'Content-Type': 'text/plain'
}

params = {
  'remotepath': '/',
  'filename': 'testCityTestSF.csv'
}

r = requests.post(
    'https://sftp-microservice-py-prod.herokuapp.com/sftp',
    files=files,
    headers=headers,
    params=params)
r.text


'{"status": "error", "message": "Cannot transfer file: No hostkey for host sftp.getcolor.com found."}'

In [47]:
key = paramiko.pkey.PKey(msg=SFTP_HOST_KEY, data=SFTP_HOST_KEY)

In [87]:
# cnopts = pysftp.CnOpts()
# cnopts.hostkeys.add(SFTP_HOSTNAME,'ssh-rsa', key)


# sftp = pysftp.Connection(SFTP_HOSTNAME, cnopts=cnopts,
#                                  username=SFTP_USERNAME, password=SFTP_PASSWORD)

# Start to finish

In [131]:
# Get Acuity appointments and parse
appointments = get_acuity_appointments((date.today() + datetime.timedelta(days=1)), (date.today() + datetime.timedelta(days=1)))
parsed_appointments = [parse_appointment(appt) for appt in appointments]

# Get and parse formio responses
dsws = [appt['dsw'] for appt in parsed_appointments if appt['dsw']]
parsed_formio_responses = dict([parse_formio_response(r) for r in get_formio_responses(dsws)])

# Merge them together on DSW
toCSV = merge_acuity_formio(parsed_appointments, parsed_formio_responses)

# Write to csv
filename = write_csv(toCSV, filename='./CityTestSF_2020-04-09.csv')

# Write 311 info to CSV too
filename_311 = to_311_csv(toCSV)

print('Color filename', filename)
print('311 filename', filename_311)
print('Total appts booked', len(toCSV))


Color filename ./CityTestSF_2020-04-09_preliminary.csv
311 filename ./CityTestSF_Appointments_311_2020-04-09.csv
Total appts booked 65


In [1]:
get_formio_responses(dsws)

NameError: name 'get_formio_responses' is not defined

In [78]:
formio_rsponses = get_formio_responses(dsw_ids=None)

In [129]:
def to_311_csv(toCSV, filename='./CityTestSF_Appointments_311_2020-04-09.csv'):
    field_map = {
        'First Name': 'firstName',
        'Last Name': 'lastName',
        'Phone Number': 'phone',
        'Email': 'email',
        'DSW': 'dsw',
        'Appointment Time': 'appointmentDatetime',
        'Appointment Date': 'appointmentDatetime',
    }
        
    mapped = []
    
    for appt in toCSV:
        mapped_appt = {k: appt[v] for k,v in field_map.items()}
        
        # format date and time nicely
        dt_str = appt['appointmentDatetime']
        dt_format ='%Y-%m-%dT%H:%M:%S%z'
        dt = datetime.datetime.strptime(dt_str, dt_format)
        mapped_appt['Appointment Date'] = dt.strftime('%m/%d/%Y')
        mapped_appt['Appointment Time'] = dt.strftime('%I:%M %p')
        
        mapped.append(mapped_appt)

    with open(filename, 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=field_map.keys(), restval='')
        dict_writer.writeheader()
        dict_writer.writerows(mapped)
    return filename

In [124]:
to_311_csv(toCSV)

filename ./CityTestSF_Appointments_311_2020-04-09.csv


'./CityTestSF_Appointments_311_2020-04-09.csv'

In [107]:
    field_map = dict({
        'First Name': 'firstName',
        'Last Name': 'lastName',
        'Phone Number': 'phone',
        'Email': 'email',
        'DSW': 'dsw',
        'Appointment Data': 'appointmentDatetime',
        'Appointment Time': 'appointmentDatetime'
    })

In [118]:
dt_str = '2020-04-09T11:00:00-0700'
dt_format ='%Y-%m-%dT%H:%M:%S%z'
dt = datetime.datetime.strptime(dt_str, dt_format)

In [120]:
dt.strftime('%I:%M %p')

'11:00 AM'

# Next steps:
1. ~~Turn json into csv~~
1. Upload to ftp server
1. Where does code live?

Clean up and improvements
1. Limit total formio query string to 2048 characters to avoid cutoff. 
1. Create a more generic dict getter to get and rename based on a map. 
1. Add tests, retry, especially to ftp server